In [19]:
import os
import numpy as np
import nibabel
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf


In [2]:
mri_files={}
seg_files={}

seg_img_root='Resized_seg_files'
mri_img_root = 'Resized_MRI_Volumes'


#creating list of files for DataGenerator
for root, dirs, files in os.walk(seg_img_root):
    for name in files:
        indx = name.find('.nii')
        key = name[indx-5:indx]
        file_path = os.path.join(root, name)
        seg_files[key] = file_path

for root, dirs, files in os.walk(mri_img_root):
    for name in files:
        indx = name.find('.nii')
        key = name[indx-5:indx]
        file_path = os.path.join(root, name)
        mri_files[key] = file_path 
        
        
        
#check that every segmentation file (ground truth) has corresponding X data
for k in seg_files.keys():
    if k not in mri_files.keys():
        print('Not found in mri files:', k)
  

In [3]:
samples_list = [k for k in seg_files.keys()]

In [31]:
class TrainDataGenerator(keras.utils.Sequence):

    def __init__(self, samples_list, y_dict, x_dict, batch_size=16, dim=(80,120,120), n_channels=4, shuffle=True):
     
        self.samples_list=samples_list
        self.y_dict=y_dict
        self.x_dict=x_dict
        self.batch_size=batch_size
        self.dim=dim
        self.n_channels=n_channels
        self.shuffle=shuffle
        self.on_epoch_end()

    def __len__(self):
        #Take all batches in each iteration'
        return int(np.floor(len(self.samples_list) / self.batch_size))

    def __getitem__(self, index):
        'Get next batch'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # single file
        samples_list_temp = [samples_list[k] for k in indexes]

        # Set of X_train and y_train
        X, y = self.__data_generation(samples_list_temp)

        return X, y

    def on_epoch_end(self):
        #Updates indexes after each epoch'
        self.indexes = np.arange(len(self.samples_list))
        #shuffle data if shuffle is true
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            

    def __data_generation(self, samples_list_temp):
        #Generates data containing batch_size samples'
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, 80,120,120,1), dtype=np.float32)
        
        # Generate data
        for i, file_key in enumerate(samples_list_temp):
            # Store sample
            X[i,] = nibabel.load(self.x_dict[file_key]).get_fdata()
            
            # Store target segmentation
            y[i,] = nibabel.load(self.y_dict[file_key]).get_fdata()

        return X, y

        
# 75-20-5 train-validation-test split of data 

#FIXME
#adjusted to partial sample list
remain_samples, test_samples, _ , _ = train_test_split(samples_list, samples_list, test_size=0.05, random_state=42)

train_samples, val_samples, _ , _ = train_test_split(remain_samples, remain_samples, test_size=0.21, random_state=42)

#build data generators

img_data_gen_args = {'batch_size':4,
                         'dim':(80,120, 120), 
                         'n_channels':4,
                         'shuffle':True}

train_data_generator = TrainDataGenerator(samples_list = train_samples,
                                          y_dict = seg_files,
                                          x_dict = mri_files,
                                          **img_data_gen_args)

val_data_generator = TrainDataGenerator(samples_list = test_samples,
                                          y_dict = seg_files,
                                          x_dict = mri_files,
                                          **img_data_gen_args)

In [15]:
import keras.backend as K

def IoU_wrapper(num_classes):
    def IoU_coef(y_true, y_pred) -> float:
        '''
        Computes Intersection-Over-Union also known as Jaccard Index
        For multiclass task computes IoU individually for each class
            and then returns average. Class labels should begin with 0

        Inputs:
        ground_truth: a numpy array of the correct labels
        y_preds: a numpy array of the predicted labels
        num_classes: an int for the number of classes

        Output:
        a float that is the average calculated IoU
        '''

        y_true = tf.math.round(y_true)
        y_pred = tf.math.round(y_pred)
        iou_array = tf.TensorArray(tf.float32, size=num_classes, clear_after_read=False)

        for c in range(num_classes):
            
            y_true_vals = tf.cast(K.flatten(tf.math.equal(y_true, tf.constant(c, tf.float32))), tf.float32)
            y_pred_vals = tf.cast(K.flatten(tf.math.equal(y_pred, tf.constant(c, tf.float32))), tf.float32)
            intersection = K.sum(y_true_vals * y_pred_vals)
            union =  K.sum(y_true_vals) + K.sum(y_pred_vals) - intersection

            iou_array = iou_array.write(c, intersection/union)
        
        return K.mean(iou_array.stack())
    return IoU_coef


def dice_wrapper(num_classes):
    def dice_coef(y_true, y_pred) -> float:
        '''
        Computes dice coefficient
        For multiclass task computes dice individually for each class
            and then returns average. Class labels should begin with 0

        Inputs:
        ground_truth: a numpy array of the correct labels
        y_preds: a numpy array of the predicted labels
        num_classes: an int for the number of classes

        Output:
        [a float that is the average dice]
        '''
        y_true = tf.math.round(y_true)
        y_pred = tf.math.round(y_pred)
        dice_array = tf.TensorArray(tf.float32, size=num_classes, clear_after_read=False)

        for c in range(num_classes):
            y_true_vals = tf.cast(K.flatten(tf.math.equal(y_true, tf.constant(c, tf.float32))), tf.float32)
            y_pred_vals = tf.cast(K.flatten(tf.math.equal(y_pred, tf.constant(c, tf.float32))), tf.float32)
            intersection = K.sum(y_true_vals * y_pred_vals)

            dice_array = dice_array.write(c, (2*intersection)/(K.sum(y_true_vals) + K.sum(y_pred_vals)))

        
        return K.mean(dice_array.stack())
    return dice_coef


def binary_IoU_coef(y_true, y_pred) -> float:
        '''
        Computes Intersection-Over-Union also known as Jaccard Index
        For binary segmentation

        Inputs:
        ground_truth: a numpy array of the correct labels
        y_preds: a numpy array of the predicted labels
        num_classes: an int for the number of classes

        Output:
        a float of calculated IoU
        '''

        y_true_vals = K.flatten(y_true)
        y_pred_vals = K.flatten(y_pred)
        intersection = K.sum(y_true_vals * y_pred_vals)
        union =  K.sum(y_true_vals) + K.sum(y_pred_vals) - intersection

    
        return intersection/union
    
def binary_dice_coef(y_true, y_pred) -> float:
        '''
        Computes dice coefficient
        For binary class segmentation

        Inputs:
        ground_truth: a numpy array of the correct labels
        y_preds: a numpy array of the predicted labels
        num_classes: an int for the number of classes

        Output:
        a float of dice coefficient]
        '''

        y_true_vals = K.flatten(y_true)
        y_pred_vals = K.flatten(y_pred)
        intersection = K.sum(y_true_vals * y_pred_vals)

        dice = (2*intersection)/(K.sum(y_true_vals) + K.sum(y_pred_vals))


        return dice

def dice_loss(y_true, y_pred):
        return (1/binary_dice_coef(y_true, y_pred)) - 1
    

In [57]:
y_true = [1,0,0]
y_pred = [1,1,1]

iou_array = tf.TensorArray(tf.float32, size=1, clear_after_read=False)

y_true_vals = tf.cast(K.flatten(tf.math.equal(y_true, tf.constant(1))), tf.float32)
y_pred_vals = tf.cast(K.flatten(tf.math.equal(y_pred, tf.constant(1))), tf.float32)
intersection = K.sum(y_true_vals * y_pred_vals)
union =  K.sum(y_true_vals) + K.sum(y_pred_vals) - intersection

iou_array = iou_array.write(0, intersection/union)

K.mean(iou_array.stack())



<tf.Tensor: shape=(), dtype=float32, numpy=0.33333334>

In [6]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, concatenate, BatchNormalization, Dense, Dropout, Flatten 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import Activation, UpSampling3D, ZeroPadding3D

#free up RAM
keras.backend.clear_session()

def conv_block(input_layer, num_filters):
    x = Conv3D(filters=num_filters, kernel_size=3, strides=1, padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv3D(filters=num_filters, kernel_size=3, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

def encoder_block(input_layer, num_filters):
    x = conv_block(input_layer, num_filters)
    out = MaxPooling3D((2,2,2))(x)
    
    return out, x

def decoder_block(input_layer, conc_layer, num_filters):
    x = conv_block(input_layer, num_filters)
    x = UpSampling3D(size=2)(x)
    out = concatenate([conc_layer, x])
    
    return out
    
def build_classifier(input_shape):

    input_layer = Input(input_shape)
    
    #encoder layers
    c1, u1 = encoder_block(input_layer, 16)
    c2, u2 = encoder_block(c1,32)
    c3, u3 = encoder_block(c2, 64)
    
    #decoder layers
    c6 = decoder_block(c3, u3, 64)
    c7 = decoder_block(c6, u2, 32)
    c8 = decoder_block(c7, u1, 16)
    
    segmentation_layer = Conv3D(filters=1, kernel_size=1, activation='sigmoid', padding='same')(c8)
    
    model = Model(input_layer, segmentation_layer, name='3D_semantic_segmentation')

    return model


#free up RAM
keras.backend.clear_session()

#build model
input_shape = (80, 120, 120, 4)
model = build_classifier(input_shape)
print(model.summary())


Model: "3D_semantic_segmentation"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     


2022-03-09 11:48:07.255901: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-09 11:48:07.619244: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-09 11:48:07.620919: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-09 11:48:07.648314: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

 input_1 (InputLayer)           [(None, 80, 120, 12  0           []                               
                                0, 4)]                                                            
                                                                                                  
 conv3d (Conv3D)                (None, 80, 120, 120  1744        ['input_1[0][0]']                
                                , 16)                                                             
                                                                                                  
 batch_normalization (BatchNorm  (None, 80, 120, 120  64         ['conv3d[0][0]']                 
 alization)                     , 16)                                                             
                                                                                                  
 activation (Activation)        (None, 80, 120, 120  0           ['batch_normalization[0][0]']    
          

                                64)                                                               
                                                                                                  
 up_sampling3d (UpSampling3D)   (None, 20, 30, 30,   0           ['activation_7[0][0]']           
                                64)                                                               
                                                                                                  
 concatenate (Concatenate)      (None, 20, 30, 30,   0           ['activation_5[0][0]',           
                                128)                              'up_sampling3d[0][0]']          
                                                                                                  
 conv3d_8 (Conv3D)              (None, 20, 30, 30,   110624      ['concatenate[0][0]']            
                                32)                                                               
          

In [16]:
#testing customs metrics
iou_metric = IoU_wrapper(2)
dice_metric = dice_wrapper(2)

model.compile(loss=dice_loss, optimizer=Adam(), metrics=['acc', iou_metric, dice_metric, 
                                                                     binary_IoU_coef, binary_dice_coef])
epochs=1
history = model.fit(train_data_generator , 
                              validation_data=val_data_generator,
                             epochs=epochs,
                             verbose=1,)

  1/908 [..............................] - ETA: 36:01 - loss: 1.6446 - acc: 0.9960 - IoU_coef: 0.8803 - dice_coef: 0.9323 - binary_IoU_coef: 0.2331 - binary_dice_coef: 0.3781

2022-03-09 11:54:03.043563: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 18432000 exceeds 10% of free system memory.


113/908 [==>...........................] - ETA: 5:46 - loss: 2.1783 - acc: 0.9930 - IoU_coef: 0.7808 - dice_coef: 0.8483 - binary_IoU_coef: 0.2714 - binary_dice_coef: 0.4112

KeyboardInterrupt: 

In [26]:
x.shape

(80, 120, 120, 4)

In [32]:

checkpoint = keras.callbacks.ModelCheckpoint('resized_MRI_segmentation_dice_loss.h5', save_weights_only=True)
early_stopping = keras.callbacks.EarlyStopping(monitor='binary_dice_coef', patience=10, mode='max')

callbacks = [checkpoint, early_stopping]
model.compile(loss=dice_loss, optimizer=Adam(), metrics=['acc', iou_metric, dice_metric, 
                                                                     binary_IoU_coef, binary_dice_coef])

epochs=30
history = model.fit(train_data_generator , 
                              validation_data=val_data_generator,
                             epochs=epochs,
                             verbose=1,
                             callbacks=callbacks)


Epoch 1/30
227/227 [==============================] - 450s 2s/step - loss: 0.4164 - acc: 0.9941 - IoU_coef: 0.7930 - dice_coef: 0.8669 - binary_IoU_coef: 0.5747 - binary_dice_coef: 0.7231 - val_loss: 1.1812 - val_acc: 0.9945 - val_IoU_coef: 0.8093 - val_dice_coef: 0.8703 - val_binary_IoU_coef: 0.5943 - val_binary_dice_coef: 0.7226
Epoch 2/30
227/227 [==============================] - 445s 2s/step - loss: 0.2829 - acc: 0.9954 - IoU_coef: 0.8309 - dice_coef: 0.8971 - binary_IoU_coef: 0.6515 - binary_dice_coef: 0.7856 - val_loss: 528.7045 - val_acc: 0.9952 - val_IoU_coef: 0.8090 - val_dice_coef: 0.8716 - val_binary_IoU_coef: 0.6030 - val_binary_dice_coef: 0.7317
Epoch 3/30
227/227 [==============================] - 445s 2s/step - loss: 0.2416 - acc: 0.9959 - IoU_coef: 0.8471 - dice_coef: 0.9087 - binary_IoU_coef: 0.6847 - binary_dice_coef: 0.8100 - val_loss: 47.3841 - val_acc: 0.9952 - val_IoU_coef: 0.8067 - val_dice_coef: 0.8643 - val_binary_IoU_coef: 0.6099 - val_binary_dice_coef: 0.726

227/227 [==============================] - 433s 2s/step - loss: 0.1052 - acc: 0.9979 - IoU_coef: 0.9168 - dice_coef: 0.9542 - binary_IoU_coef: 0.8305 - binary_dice_coef: 0.9063 - val_loss: 0.1945 - val_acc: 0.9977 - val_IoU_coef: 0.8976 - val_dice_coef: 0.9388 - val_binary_IoU_coef: 0.7919 - val_binary_dice_coef: 0.8754
Epoch 26/30
227/227 [==============================] - 437s 2s/step - loss: 0.1095 - acc: 0.9978 - IoU_coef: 0.9136 - dice_coef: 0.9523 - binary_IoU_coef: 0.8243 - binary_dice_coef: 0.9027 - val_loss: 0.1364 - val_acc: 0.9979 - val_IoU_coef: 0.9094 - val_dice_coef: 0.9474 - val_binary_IoU_coef: 0.8151 - val_binary_dice_coef: 0.8924
Epoch 27/30
227/227 [==============================] - 442s 2s/step - loss: 0.1081 - acc: 0.9978 - IoU_coef: 0.9142 - dice_coef: 0.9527 - binary_IoU_coef: 0.8256 - binary_dice_coef: 0.9036 - val_loss: 0.1447 - val_acc: 0.9974 - val_IoU_coef: 0.9066 - val_dice_coef: 0.9458 - val_binary_IoU_coef: 0.8100 - val_binary_dice_coef: 0.8895
Epoch 28/3

KeyboardInterrupt: 

In [33]:
import pickle

In [35]:
with open('trainHistoryDict', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

NameError: name 'history' is not defined

In [36]:
history

import os
import numpy as np
import nibabel
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
mri_files={}
seg_files={}

seg_img_root='Resized_seg_files'
mri_img_root = 'Resized_MRI_Volumes'


#creating list of files for DataGenerator
for root, dirs, files in os.walk(seg_img_root):
    for name in files:
        indx = name.find('.nii')
        key = name[indx-5:indx]
        file_path = os.path.join(root, name)
        seg_files[key] = file_path

for root, dirs, files in os.walk(mri_img_root):
    for name in files:
        indx = name.find('.nii')
        key = name[indx-5:indx]
        file_path = os.path.join(root, name)
        mri_files[key] = file_path 
        
        
        
#check that every segmentation file (ground truth) has corresponding X data
for k in seg_files.keys():
    if k not in mri_files.keys():
        print('Not found in mri files:', k)
samples_list = [k for k in seg_files.keys()]
class TrainDat